In [527]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [540]:
# Convert to UTF-8 as follows:
# iconv -c -f ASCII -t UTF-8 < Trade_DetailedTradeMatrix_E_All_Data_\(Normalized\).csv > trade.csv &

trade_raw = pd.read_csv('Data/trade.csv')
food_balance_raw = pd.read_csv('Data/balance.csv')
food_supply_raw = pd.read_csv('Data/supply.csv')
food_production_raw = pd.read_csv('Data/production.csv')
malnutrition_raw = pd.read_csv('Data/malnutrition.csv')
population_raw = pd.read_csv('Data/population.csv')

/Users/sean/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0,2,4,6,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [649]:
# CONSTANTS
PREDICTION_LENGTH = 1

In [637]:
# Filter Food Production to only include production (Element Code 5510, tonnes)
food_production_unaggregated = food_production_raw.loc[:,['Area Code','Item Code','Element Code','Year Code','Value']].loc[food_production_raw['Element Code']==5510].drop(columns=['Element Code']).apply(pd.to_numeric, errors='ignore')
# Filter Food Supply to only include Food Supply Quantity (Element Code 641, tonnes)
food_supply = food_supply_raw.loc[:,['Area Code','Item Code','Element Code','Year Code','Value']].loc[food_supply_raw['Element Code']==641].drop(columns=['Element Code']).apply(pd.to_numeric, errors='ignore')
# Filter Food Balance to only include food available for consumption (Element Code 5142, 1000 tonnes) -> need to multiply by 1000 to get units in tonnes
food_balance = food_balance_raw.loc[:,['Area','Item','Area Code','Item Code','Element Code','Year Code','Value']].loc[food_balance_raw['Element Code']==5142].drop(columns=['Element Code']).apply(pd.to_numeric, errors='ignore')
food_balance.loc[:,['Value']] = food_balance.loc[:,['Value']].values * 1000.
# Filter Trade to only include export quantity (Element Code 5910, tonnes) and import quantity (Element Code 5610, tonnes)
# Note that reporter and partner quantity may not agree (http://fenixservices.fao.org/faostat/static/documents/TM/TM_e.pdf)
# Thus only count imports and exports with respect to the reporting country.  That's what matter for actual balance.
trade_export_unaggregated = trade_raw.loc[:,['Reporter Country Code','Item Code','Element Code','Year Code','Value']].loc[trade_raw['Element Code']==5910].rename(columns={'Reporter Country Code':'Area Code'}).drop(columns=['Element Code']).apply(pd.to_numeric, errors='ignore')
trade_import_unaggregated = trade_raw.loc[:,['Reporter Country Code','Item Code','Element Code','Year Code','Value']].loc[trade_raw['Element Code']==5610].rename(columns={'Reporter Country Code':'Area Code'}).drop(columns=['Element Code']).apply(pd.to_numeric, errors='ignore')
# Filter Population to only include Total Population- both sexes (Element Code 511, 1000 persons) -> need to multiply by 1000 to get number of people
population = population_raw.loc[:,['Area Code','Year Code','Value','Element Code']].loc[population_raw['Element Code']==511].rename(columns={'Value': 'Population'}).drop(columns=['Element Code']).apply(pd.to_numeric, errors='ignore')
population.loc[:,['Population']] = population.loc[:,['Population']].values * 1000.

In [638]:
# DEBUG - only consider the USA
# food_production_unaggregated = food_production_unaggregated.loc[food_production_unaggregated['Area Code']==231]
# food_supply = food_supply.loc[food_supply['Area Code']==231]
# food_balance = food_balance.loc[food_balance['Area Code']==231]
# trade_unaggregated = trade_unaggregated.loc[(trade_unaggregated['Area Code']==231)]
# DEBUG - only consider the USA

In [639]:
# FAO aggregates products for Food Balance and Food Supply
# Food Production and Trade is not aggregated
# In order to compare the datasets, Food Production and Trade must be aggregated using the aggregate mapping
# The aggregate mapping requires pre-processing
aggregate_mapping_raw = pd.read_csv('Data/aggregate-mapping.csv').loc[:,['Item Code', 'Description']].rename(columns={'Item Code':'Aggregate Item Code'})
aggregate_index_mapping = aggregate_mapping_raw.Description.str.extractall('(\d+)').reset_index().drop(columns=['match']).rename(columns={0:'Item Code'}).rename(columns={'level_0':'Aggregated Item Code Index'})
aggregate_mapping = pd.merge(left=aggregate_mapping_raw,
                             right=aggregate_index_mapping,
                             how='outer',
                             left_index=True,
                             right_on=['Aggregated Item Code Index']).dropna().drop(columns=['Aggregated Item Code Index','Description']).apply(pd.to_numeric, errors='ignore')

In [658]:

# Aggregate the Food Production and Trade dataframes
food_production = pd.merge(left=food_production_unaggregated,
                                      right=aggregate_mapping,
                                      how='inner',
                                      on='Item Code').drop(columns=['Item Code']).rename(columns={'Aggregate Item Code':'Item Code'})
trade_import = pd.merge(left=trade_import_unaggregated,
                            right=aggregate_mapping,
                            how='inner',
                            on='Item Code').drop(columns=['Item Code']).rename(columns={'Aggregate Item Code':'Item Code'})
trade_export = pd.merge(left=trade_export_unaggregated,
                            right=aggregate_mapping,
                            how='inner',
                            on='Item Code').drop(columns=['Item Code']).rename(columns={'Aggregate Item Code':'Item Code'})

In [668]:
print(len(food_production))
print(len(food_production_unaggregated))
print(len(food_production_unaggregated.groupby(['Area Code', 'Item Code', 'Year Code']).sum().reset_index()))
food_production_unaggregated.head()

726787
1327954
1316524


,Area Code,Item Code,Year Code,Value
0,2,767,1961,17000.0
1,2,767,1962,26000.0
2,2,767,1963,36700.0
3,2,767,1964,26000.0
4,2,767,1965,25000.0


In [641]:
# Given [country, year, item code, element code]
# Predict [future balance]
# From [supply, amount traded out, amount traded in]

# Create new prediction_data dataframe from food_supply dataframe
prediction_data = food_supply.copy().rename(columns={'Value':'Current Supply'})
print(len(prediction_data))

1136394


In [642]:
# Rename the [Value] column of food_production to be [Amount Produced]
food_production = food_production.rename(columns={'Value': 'Amount Produced'})
# Merge food_balance with prediction_data on [Area Code, Item Code, 'Year Code']
prediction_data = pd.merge(left=prediction_data,
                          right=food_production,
                          how='inner',
                          on=['Area Code','Item Code','Year Code'])
print(len(prediction_data))

615762


In [643]:
# Filter by summing [Value] for a constant [Area Code, Item Code, Year Code]
trade_export = trade_export.groupby(['Area Code', 'Item Code', 'Year Code'])[['Value']].sum().reset_index()
# Rename the [Value] column of trade_export to be [Amount Exported]
trade_export = trade_export.rename(columns={'Value': 'Amount Exported'})
# Merge trade_export with prediction_data on [Area Code, Item Code, Year Code]
prediction_data = pd.merge(left=prediction_data,
                           right=trade_export,
                           how='inner',
                           on=['Area Code', 'Item Code', 'Year Code'])
print(len(prediction_data))

160679


In [644]:
# Filter by summing [Value] for a constant [Area Code, Item Code, Year Code]
trade_import = trade_import.groupby(['Area Code', 'Item Code', 'Year Code'])[['Value']].sum().reset_index()
# Rename the [Value] column of trade_import to be [Amount Exported]
trade_import = trade_import.rename(columns={'Value': 'Amount Imported'})
# Merge trade_import with prediction_data on [Area Code, Item Code, Year Code]
prediction_data = pd.merge(left=prediction_data,
                           right=trade_import,
                           how='inner',
                           on=['Area Code', 'Item Code', 'Year Code'])
print(len(prediction_data))

155370


In [645]:
# Insert population into the prediction_data
prediction_data = pd.merge(left=prediction_data,
                          right=population,
                          how='left',
                          on=['Area Code','Year Code'])
print(len(prediction_data))

155370


In [646]:
# Translate the food_balance out PREDICTION_LENGTH years
food_balance.loc[:,['Year Code']] = food_balance.loc[:,['Year Code']].values - PREDICTION_LENGTH
# Rename the [Value] column of food_balance to be [Predicted Value]
food_balance = food_balance.rename(columns={'Value': 'Predicted Balance'})
# Merge food_balance with prediction_data on [Area Code, Item Code, 'Year Code']
prediction_data = pd.merge(left=prediction_data,
                          right=food_balance,
                          how='inner',
                          on=['Area Code','Item Code','Year Code'])
print(len(prediction_data))

145544


In [636]:
# Input Data Columns: [Area Code, Year Code, Item Code, Value, Value In, Value Out]
# Predicted Data Columns: [Predicted Value]
# prediction_data.drop(columns=['Item','Area','Area Code',Year Code'])
prediction_data

,Area Code,Item Code,Year Code,Current Supply,Amount Produced,Amount Exported,Amount Imported,Population,Area,Item,Predicted Balance
0,3,2656,1999,34940.00,8703.0,6.0,26244.0,3115576.0,Albania,Beer,41000.0
1,3,2656,2001,47929.00,11670.0,24.0,36286.0,3122408.0,Albania,Beer,43000.0
2,3,2656,2002,43297.00,15470.0,32.0,27857.0,3119029.0,Albania,Beer,40000.0
3,3,2656,2003,40192.00,14400.0,66.0,25859.0,3111005.0,Albania,Beer,52000.0
4,3,2656,2004,51969.00,29830.0,287.0,22425.0,3097747.0,Albania,Beer,49000.0
5,3,2656,2005,48643.00,28630.0,130.0,20145.0,3079179.0,Albania,Beer,57000.0
6,3,2656,2006,57215.00,34800.0,509.0,22925.0,3054331.0,Albania,Beer,66000.0
7,3,2656,2007,66039.00,36500.0,577.0,30116.0,3023907.0,Albania,Beer,66000.0
8,3,2656,2008,66049.00,33000.0,417.0,33465.0,2991651.0,Albania,Beer,61000.0
9,3,2656,2009,61463.00,24900.0,219.0,36783.0,2962635.0,Albania,Beer,78000.0
